In [6]:
import json
from collections import Counter
import pandas as pd

In [21]:
# JSON 파일 불러오기
with open("../data/final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# JSON을 DataFrame으로 변환
df = pd.DataFrame(data)

# 데이터 개수 확인
print("📌 총 데이터 개수:", len(df))

# 특정 컬럼(예: 'title') 기준 중복 확인
duplicate_titles = df[df.duplicated(subset=['title'], keep=False)]
print("\n📌 'title' 기준 중복된 행 개수:", len(duplicate_titles))



📌 총 데이터 개수: 841

📌 'title' 기준 중복된 행 개수: 0


In [22]:
df = df.sort_values(by=["date"])

In [24]:
def save_json(df, file_path):
    json_data = json.loads(df.to_json(orient="records", force_ascii=False))
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

save_json(df, "../data/final.json")

### 분기별 개수

In [ ]:
from collections import defaultdict
import os
# JSON 파일이 저장된 디렉토리 경로
output_dir = "../data/split_by_year"

# 분기별 데이터 개수 저장 딕셔너리
quarterly_counts = defaultdict(int)

# 각 JSON 파일에서 데이터 개수 확인
for file_name in os.listdir(output_dir):
    if file_name.endswith(".json"):
        year = file_name.replace(".json", "")  # 파일명에서 연도 추출
        file_path = os.path.join(output_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

            # 날짜 필드에서 분기별로 데이터 개수 계산
            for entry in data:
                if "date" in entry:
                    try:
                        quarter = pd.to_datetime(entry["date"]).to_period("Q")  # 분기 추출
                        quarterly_counts[str(quarter)] += 1
                    except:
                        print(f"날짜 변환 실패: {entry['date']}")

# 분기별 데이터 개수 출력
for quarter, count in sorted(quarterly_counts.items()):
    print(f"{quarter}: {count}개")

### 연도별 개수

In [ ]:
# 연도별 데이터 개수 저장 딕셔너리
yearly_counts = defaultdict(int)

# 각 JSON 파일에서 데이터 개수 확인
for file_name in os.listdir(output_dir):
    if file_name.endswith(".json"):
        year = file_name.replace(".json", "")  # 파일명에서 연도 추출
        file_path = os.path.join(output_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            yearly_counts[year] = len(data)

# 연도별 데이터 개수 출력
for year, count in sorted(yearly_counts.items()):
    print(f"{year}: {count}개")

### 토큰 수

In [31]:
import tiktoken

# JSON 파일 불러오기
with open("../data/final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# JSON을 DataFrame으로 변환
df = pd.DataFrame(data)

# OpenAI GPT-3.5 토크나이저 로드
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

# 'sentences' 컬럼과 'filtered_sentences' 컬럼의 토큰 수 계산
df["sentences_token_count"] = df["sentences"].apply(lambda x: sum(len(enc.encode(sentence)) for sentence in x) if isinstance(x, list) else 0)
df["filtered_sentences_token_count"] = df["filtered_sentences"].apply(lambda x: sum(len(enc.encode(sentence)) for sentence in x) if isinstance(x, list) else 0)

# 결과 출력
print(df[["sentences_token_count", "filtered_sentences_token_count"]])


     sentences_token_count  filtered_sentences_token_count
0                      590                              64
1                     1215                              56
2                     2826                             333
3                     2843                             326
4                     1328                              54
..                     ...                             ...
836                    380                              71
837                    370                              71
838                    380                              71
839                    390                              38
840                   2112                              79

[841 rows x 2 columns]


In [34]:
print(df[["sentences_token_count", "filtered_sentences_token_count","num_sentences"]].describe())

       sentences_token_count  filtered_sentences_token_count  num_sentences
count             841.000000                      841.000000     841.000000
mean             1654.387634                      161.021403      28.346017
std              1392.411530                      195.585853      25.573012
min                 0.000000                        0.000000       0.000000
25%               738.000000                       55.000000      13.000000
50%              1465.000000                      102.000000      24.000000
75%              2127.000000                      193.000000      37.000000
max             20338.000000                     1635.000000     363.000000


In [33]:
df

,title,link,date,content,quarter,cleaned_content,sentences,filtered_sentences,num_filtered_sentences,num_sentences,ratio,sentences_token_count,filtered_sentences_token_count
0,朝鲜试射导弹亚太骤起波澜,https://www.gmw.cn/01gmrb/2006-07/12/content_4...,2006-07-12,7月5日，朝鲜连续发射7枚导弹，引起国际社会震惊。近日，美日在安理会积极推动制裁朝鲜的议案，...,2006Q3,7月5日朝鲜连续发射7枚导弹引起国际社会震惊。近日美日在安理会积极推动制裁朝鲜的议案并加紧协...,"[7月5日朝鲜连续发射7枚导弹引起国际社会震惊。, 近日美日在安理会积极推动制裁朝鲜的议案并...",[朝鲜发射导弹后美日韩澳俄英法等国纷纷发表声明称朝鲜此举是挑衅行为破坏了亚太的稳定。],1,15,0.066667,590,64
1,美朝关系开始解冻,https://www.gmw.cn/01gmrb/2007-03/08/content_5...,2007-03-08,被外界视为“破冰之旅”的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常...,2007Q1,被外界视为破冰之旅的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常化进...,[被外界视为破冰之旅的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常化...,[另一方面在朝鲜近期对美日韩等国及国际原子能机构所采取的一系列外交攻势下美国的态度也有所变化。],1,20,0.050000,1215,56
2,美日韩间谍云集朝鲜周边空中侦察地面窃听 - 国际经济,http://intl.ce.cn/zj/200809/19/t20080919_16855...,2008-09-19,神秘，这是朝鲜留给外界的印象。 正因为神秘，一些国家的情报机构费尽心机，通过投放卫星、派遣侦...,2008Q3,神秘这是朝鲜留给外界的印象。 正因为神秘一些国家的情报机构费尽心机通过投放卫星派遣侦察机甚至...,"[神秘这是朝鲜留给外界的印象。, 正因为神秘一些国家的情报机构费尽心机通过投放卫星派遣侦察机...",[美日韩编织谍报大网 朝鲜地缘位置敏感环境又相对封闭以至朝鲜许多本应正常公开的信息都成为这些...,6,52,0.115385,2826,333
3,美日韩紧盯朝鲜射导弹(图) - 国际经济,http://intl.ce.cn/zj/200902/11/t20090211_18171...,2009-02-11,改进型“大浦洞2号”据说可攻击美本土，甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关...,2009Q1,改进型大浦洞2号据说可攻击美本土甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关系恶化...,[改进型大浦洞2号据说可攻击美本土甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关系恶...,[一直在密切注意朝鲜动向的美日韩三国政府近日搜集到朝鲜可能发射远程导弹的情报这可急坏了三国的...,6,50,0.120000,2843,326
4,朝鲜要用150天救经济摆脱美日韩经济封锁,https://world.huanqiu.com/article/9CaKrnJlXnI,2009-05-20,自上个月朝鲜宣布发射人造卫星以来，美国、日本和韩国高举制裁大棒，对朝鲜的贸易活动进行了进一步...,2009Q2,自上个月朝鲜宣布发射人造卫星以来美国日本和韩国高举制裁大棒对朝鲜的贸易活动进行了进一步的封锁...,[自上个月朝鲜宣布发射人造卫星以来美国日本和韩国高举制裁大棒对朝鲜的贸易活动进行了进一步的封...,[朝鲜宣布退出六方会谈并重启核设施美日韩三国对朝关系重现紧张态势相关的援助也相应中断。],1,22,0.045455,1328,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,朝鲜谴责美韩军事挑衅导致地区局势恶化 - 国际经济,http://intl.ce.cn/qqss/202501/26/t20250126_392...,2025-01-26,新华社首尔1月26日电 据朝中社26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期...,2025Q1,新华社首尔1月26日电 据朝中社26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝...,[新华社首尔1月26日电 据朝中社26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对...,[谈话说美韩近期针对朝鲜进行联合空中训练联合大火力战演习年初还在战略轰炸机参与下启动美日韩联...,1,7,0.142857,380,71
837,朝鲜谴责美韩军事挑衅导致地区局势恶化 - 国际频道,https://world.gmw.cn/2025-01/26/content_378203...,2025-01-26,据朝中社1月26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期对朝军事挑衅行为导致...,2025Q1,据朝中社1月26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝军事挑衅行为导致地区...,[据朝中社1月26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝军事挑衅行为导致地...,[谈话说美韩近期针对朝鲜进行联合空中训练联合大火力战演习年初还在战略轰炸机参与下启动美日韩联...,1,7,0.142857,370,71
838,朝鲜谴责美韩军事挑衅导致地区局势恶化,http://www.xinhuanet.com/20250126/55a39b230ccf...,2025-01-26,新华社首尔1月26日电 据朝中社26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期...,2025Q1,新华社首尔1月26日电 据朝中社26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝...,[新华社首尔1月26日电 据朝中社26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对...,[谈话说美韩近期针对朝鲜进行联合空中训练联合大火力战演习年初还在战略轰炸机参与下启动美日韩联...,1,7,0.142857,380,71
839,朝鲜外务省：美韩军事挑衅行为导致地区局势恶化 - 国际频道,https://world.gmw.cn/2025-01/26/content_378203...,2025-01-26,中新网1月26日电据朝中社报道，当地时间1月26日，朝鲜外务省对外政策室长发表谈话，谴责美韩...,2025Q1,中新网1月26日电据朝中社报道当地时间1月26日朝鲜外务省对外政策室长发表谈话谴责美韩近期对...,[中新网1月26日电据朝中社报道当地时间1月26日朝鲜外务省对外政策室长发表谈话谴责美韩近期...,[2025年年初还在战略轰炸机参与下启动美日韩三方联合空中训练。],1,8,0.125000,390,38
